# Victim-Protagonist Representation Across Various Guatemalan Truth Initiatives - An LLM and Human-in-the-Loop Analysis

# Initialization of the packages

In [1]:
#Call the libraries
import pandas as pd
import openai
from openai import OpenAI
import numpy as np
import subprocess
import math
import tempfile
import wave
import os
import re
import glob
import textract

#We define which model to use throughout
MODEL = "gpt-5"
MAX_TOKENS = 2000
WAIT_TIME = 0.8 # Wait time between each request. 

# API key:
client = OpenAI(                    
    api_key="..."              
)

def low_beep(freq=220, duration=0.4, volume=0.5):
    sample_rate = 44100
    samples = int(sample_rate * duration)

    with tempfile.NamedTemporaryFile(suffix=".wav") as f:
        with wave.open(f.name, "w") as wf:
            wf.setnchannels(1)
            wf.setsampwidth(2)
            wf.setframerate(sample_rate)

            for i in range(samples):
                value = int(volume * 32767 *
                            math.sin(2 * math.pi * freq * i / sample_rate))
                wf.writeframes(value.to_bytes(2, "little", signed=True))

        subprocess.run(["afplay", f.name], check=False)

# Prompt engineering

In [62]:
instruction_english_part1 = """You are a social scientist in the domain of transitional justice that has the task to classify extracts of truth commission reports.

Determine whether the segment talks about the victims or whether the victims themselves speak or testify using the ‘victim presence’ coding “0”: victims are not mentioned; “1”: the segment talks about the victims; ‘2’: the victims speak/testify themselves; “3”: BOTH, the segment talks about the victims and the victims speak/testify themselves in the segment.

The format of the result is divided into 2 parts separated by a semicolon (;):
integer value representing the victim as described above, followed by a semicolon (;),
and then a brief explanation in maximum 50 tokens. Do not use quotation marks.
IMPORTANT NOTE: if you determined that ‘victim presence’ coding is “0”: victims are not mentioned. then the return should be 2 comma-separated zeroes (0)"""

instruction_spanish_part1 = """Eres un científico social en el ámbito de la justicia transicional crítica, y tienes la tarea de clasificar extractos de los informes de diferentes comisiones de verdad y de esclarecimiento histórico:

Determine si el segmento explicitamente habla de las víctimas o si son las propias víctimas las que hablan o testifican utilizando la codificación «1»: el segmento habla explicitamente de las víctimas; «2»: las víctimas hablan/testifican ello/as mismo/as; «3»: AMBOS, el segmento habla explitamente de las víctimas y las víctimas hablan/testifican ellas mismas en el segmento; «0»: no se menciona a las víctimas explicitamente.

El formato del resultado que se debe proporcionar se divide en 2 partes separadas por un punto y coma (;):

valor entero que representa a la víctima como se ha descrito anteriormente, seguido de un punto y coma (;), 
y luego una breve explicación de 20 palabras como máximo. No utilice comillas.

NOTA IMPORTANTE: si ha determinado que la codificación de «presencia de víctimas» es «0»: no se menciona a las víctimas. Entonces, el resultado debe ser 2 ceros (0) separados por punto y coma.

A continuación se presentan algunos ejemplos sobre la codificación de la presencia de víctimas:

Segmento de ejemplo: La CEH ha formulado en el presente Informe las recomendaciones que ha juzgado necesarias para que en Guatemala se consolide la paz y la concordia nacional. Tomó en consideración los términos que en el apartado de “Finalidades” establecen el Acuerdo de Oslo y las peticiones formuladas en otros de los Acuerdos de Paz para que recomiende medidas de reparación a las víctimas, según se ha expresado anteriormente.	
Codificación esperada: 0

Segmento de ejemplo: La formación de un Estado democrático se ha visto limitada permanentemente por esos condicionamientos de la historia. Su función ha consistido en mantener y conservar las estructuras de poder basadas en la explotación y la exclusión de los indígenas y de los mestizos empobrecidos. Fue necesario esperar los Acuerdos de Paz, concluidos en 1996, ciento setenta y cinco años después de la Independencia, para que el Estado guatemalteco pudiera perfilarse como un “Estado multiétnico, pluricultural y multilingüe” y, con ello, responder a una concepción de nación integradora, respetando la diversidad de sus ciudadanos.
Codificación esperada: 1

Segmento de ejemplo: La formación de un Estado democrático se ha visto limitada permanentemente por esos condicionamientos de la historia. Su función ha consistido en mantener y conservar las estructuras de poder basadas en la explotación y la exclusión de los indígenas y de los mestizos empobrecidos. Fue necesario esperar los Acuerdos de Paz, concluidos en 1996, ciento setenta y cinco años después de la Independencia, para que el Estado guatemalteco pudiera perfilarse como un “Estado multiétnico, pluricultural y multilingüe” y, con ello, responder a una concepción de nación integradora, respetando la diversidad de sus ciudadanos.
Codificación esperada: 1

Segmento de ejemplo: El rasgo característico de este momento no siempre fue la muerte del opositor, sino la creación de un clima social de inseguridad generalizada. El efecto fue el miedo: miedo a ser denunciado, a perder el empleo, a no poder retornar al país, a participar en política, a organizarse para exigir derechos. En fin, a funcionar fuera del marco de una sociedad democrática de derecho, por la existencia de un régimen de excepción permanente.
Codificación esperada: 1

Segmento de ejemplo: Se calificaba así a los campesinos que en los años anteriores se habían organizado y que habían accedido a la tierra. En la concepción del nuevo régimen el agrarismo era sinónimo de comunismo, y los beneficiarios de la ley eran asumidos como comunistas. “ ...Al poco tiempo habíamos sembrado con mi papá, cuando comenzó a ponerse las cosas un poco feo ... decían que Arbenz no iba a dilatar, y ciertamente, pues la milpa estaba en elote cuando él cayó. La gente que estaba con el patrón nos acusaba que nosotros éramos unos comunistas ... entonces yo tuve que huir”.76 “ ...En la finca Caobanal en 1954, cuando entró Castillo Armas ... todos los que habían apoyado antes a Arbenz tuvieron que irse inmediatamente, porque mandaron a quemar las casas donde vivían con todo y los animales adentro ... con buena suerte se salvó mi familia”.
Codificación esperada: 3

Segmento de ejemplo: En un mensaje episcopal del 9 de mayo de 1967 los obispos guatemaltecos manifestaron su preocupación por la creciente ola de terror que sufría el país: “Cada día aumenta el número de huérfanos y viudas, son muchos los hombres arrancados violentamente de sus hogares por incógnitos secuestradores, son detenidos en lugares ignorados o violentamente asesinados apareciendo luego sus cadáveres horriblemente destrozados y profanados. Nos angustiamos con nuestro pueblo noble y pacífico, que, desde hace ya mucho tiempo, vive la zozobra, el temor y la angustia; la inseguridad se ahonda más y más”.
Codificación esperada: 1

Segmento de ejemplo: Los efectos socioeconómicos y políticos provocados por el terremoto fueron importantes para la toma de conciencia de la población, constituyéndose en un importante elemento movilizador de la organización social, especialmente en las regiones mayas en donde se evidenciaron más las desigualdades existentes en el país, como lo expresa un dirigente campesino k’iche’: “Nosotros ya manejábamos unos elementos de la realidad nacional, pero el terremoto nos abrió los ojos, es decir, el terremoto vino a poner al desnudo nuestra realidad ... nos dábamos cuenta que la magnitud de la pobreza en el país era de tal modo que con láminas y casas prefabricadas no resolvíamos el asunto”.
Codificación esperada: 2

Segmento de ejemplo: En general, las relaciones entre la guerrilla y la población indígena tuvieron un carácter complejo y no unidireccional. Como ya se destacó en secciones precedentes de este capítulo, estas relaciones pusieron de manifiesto un interés compartido: la unión de los ladinos pobres con los mayas, los excluidos de siempre. Desde la perspectiva de la población maya hubo múltiples factores mediante los cuales se expresó este interés, como revelan las siguientes opiniones.
Codificación esperada: 1

Segmento de ejemplo: Paralelamente a los intentos de reactivación económica, el Gobierno emprendió una brutal campaña represiva contra el movimiento social, tanto en el área rural como en la urbana. El asesinato y la desaparición sistemática de líderes renombrados, así como las masacres de campesinos en el interior del país tuvo fuerte repercusión en el ámbito internacional. Guatemala se convirtió en un objeto de frecuentes sanciones y del aislamiento internacional por la intensidad de la represión estatal.
Codificación esperada: 0

Segmento de ejemplo: El movimiento social que había alcanzado un gran desarrollo en los primeros dos años del Gobierno de Laugerud García había sufrido importantes golpes por las acciones represivas que se endurecieron después del terremoto de 1976. Esto derivó en una politización y radicalización de las organizaciones sociales, tanto urbanas como rurales, al compás de su gradual movilización. Este proceso de radicalización fue resultado de un cúmulo de factores donde influyeron, además de la violencia represiva, la continuación de las exclusiones sociales y políticas, el trabajo ideológico de la guerrilla y la influencia externa de los éxitos del Frente Sandinista de Liberación Nacional (FSLN), en Nicaragua, y del Frente Farabundo Martí para la Liberación Nacional (FMLN), en El Salvador.
Codificación esperada: 1

Segmento de ejemplo: En este marco, el Ejército percibía la participación indígena ya sea en el movimiento social o insurgente como producto de su falta de integración al Estado y de un débil sentido nacionalista. A su vez, en buena medida alimentado por preceptos racistas, argumentaba que por inmadurez los indígenas eran fácilmente manipulables por la acción política de la guerrilla. El Ejército consideraba que la participación indígena y campesina estaba determinada por la presión y amenaza guerrillera, así como por el resentimiento y el abandono en que se encontraban. En consecuencia, definían su conexión con los insurgentes como potencialmente peligrosa.330 Según explicó Francisco Bianchi, portavoz de Ríos Montt en 1982, en una controvertida entrevista: “Los guerrilleros conquistaron muchos colaboradores indígenas, entonces los indígenas eran subversivos, no. ¿Y, como se lucha en contra de la insurgencia? Netamente, tendría que matar a los indígenas porque ellos estaban colaborando con la subversión”.
Codificación esperada: 1

Segmento de ejemplo: bana y rural. Todo ello explica entonces el interés de los militares en los siguientes años por darle a la política contrainsurgente un carácter de esfuerzo estatal y por institucionalizar muchos de sus procedimientos. Como indicó el mismo Ríos Montt: “Naturalmente, si una operación subversiva existe donde los indígenas están involucrados con la guerrilla, los indígenas morirán. Sin embargo no es la filosofía del Ejército matar indígenas, sino reconquistarlos y ayudarlos”.
Codificación esperada: 1

Segmento de ejemplo: En el mismo período surgió el Grupo de Apoyo Mutuo por el Aparecimiento de Familiares Desaparecidos (GAM),346 organización que jugó un importante papel en el marco de silencio que existía en el país sobre víctimas del enfrentamiento. De esta forma los planteamientos y demandas por las reivindicaciones laborales de corte tradicional y otras vinculadas a los efectos sociales del enfrentamiento pasaron a primer plano. En los primeros meses de 1985 se produjeron protestas de los universitarios, el GAM, el magisterio, sindicatos estatales y organizaciones de apoyo a los consumidores. En ese año los sindicatos volvieron a conmemorar el 1o. de mayo, por primera vez desde 1980 cuando se habían producido saldos fatales.
Codificación esperada: 1

Segmento de ejemplo: Los cambios que se produjeron en la cúpula del Ejército entre junio y septiembre fueron interpretados como expresiones de problema internos, de descontentos hacia las acciones gubernativas o como medidas previsoras de los altos mandos para evitar nuevos movimientos golpistas y mantener la línea de la estabilidad nacional. En junio, fue asesinado en Petén el estadounidense Michael Devine, por lo que la Embajada de su país presionó por que se investigara el hecho. Asimismo, en septiembre y octubre fueron asesinados la antropóloga Myrna Mack,388 cuyo trabajo de investigación se centraba en las condiciones de la población desplazada, y el político Humberto González Gamarra, dirigente socialdemócrata que buscaba ampliar espacios de participación política para sectores de izquierda.
Codificación esperada: 1

Segmento de ejemplo: El proceso de reactivación del movimiento social en el campo se dio con características similares, aunque no igualmente movilizadoras. El CUC reapareció luego de varios años en la clandestinidad al ser considerado un brazo político de la guerrilla. Más que apoyar la discusión de la tenencia de la tierra, que otros grupos planteaban, buscaba la movilización de los trabajadores agrícolas cañeros y cafetaleros en pro de mejores condiciones de trabajo, en especial en relación con el salario mínimo, además de incluir las demandas de los sectores campesinos afectados por el enfrentamiento.401 De esta forma inició tanto una lucha reivindicativa como de legitimación de su presencia política. Sus demandas crearon tensión con el Gobierno y la UNAGRO, sin que se produjeran resultados concretos, pues se concentraron en mantener la discusión en un órgano de concertación oficial y técnico sobre la posibilidad o no de incrementar salarios.
Codificación esperada: 1"""

instruction_english_part2 = """You are a social scientist in the field of critical transitional justice, and you have been tasked with classifying excerpts from the reports of different truth and historical clarification commissions, identifying when and where the following six narrative frameworks are present:

*Citizenship*, this theme can be characterized by the following dimensions (not all of them need to be present!): "rights holders", "subjects of rights", "state obligations","state responsibilities","inclusion in the democratic state","democratization","(non)recognition", "discrimination", "(social) exclusion", "marginalization", "inequality".
*Suffering*: this theme can be characterized by the following dimensions (not all of them need to be present!): "vulnerability","innocent victim(s)","deserving victim(s)","heroic victim(s)","passive victim(s)", "martyr(s)", "continuous violence", "everyday violence", "historical violence", "structural violence", "need for support", "over-victimization".
*Capacity for action*, this theme can be characterized by the following dimensions (not all of them need to be present!): "human rights advocacy","human rights defender", "resistance (everyday)", "protest", "struggle", "(community) mobilization", "resilience", "resistance", "survivor", "mutual support group(s)", "solidarity movement(s)", "protagonist", "leader", "organizer", "agents of change", "commemoration", "activist", "defender of memory".
*Reconciliation*, this theme can be characterized by the following dimensions (not all of them need to be present!): "reconciliation", "future orientation", "forgiveness", "forgetting".
*Victim blaming*, This theme can be identified when the actions of victims (such as protest) are discussed just before or after the mention of a violent act, particularly when the word ‘responsibility’ appears in relation to the victimized person or group.
*Victims' feelings often considered unacceptable*. This theme can be identified by references to victims' feelings such as "anger", "hatred", "shame", "jealousy", or "(fantasies of) revenge" (not all of these need to be present!).

For this segment, it has already been determined whether the segment explicitly talks about the victims or whether the victims themselves speak or testify using the coding “1”: the segment explicitly talks about the victims; ‘2’: the victims speak/testify themselves; “3”: BOTH, the segment explicitly talks about the victims and the victims speak/testify themselves in the segment; “0”: victims are not explicitly mentioned. The ‘victim presence’ code for this segment is {}.
First, rate the presence of each of the six narrative frames above in the segment with 0: not present, 0.5: indirectly present, 1: strongly present.
Second, provide the reasoning for arriving at the three most prevalent frames in a maximum of 200 words.
Third, provide a list of the dimensions that led you to classify all the narrative frames in the segment as defined above, 1 list for when the segment talks about the victims and 1 list for when the victims themselves speak in the segment. Note that you only choose from the dimensions that are mentioned between quotes (") after each of the six frame mentioned previously. For example, if you classify a segment as *Reconciliation*, you can only choose from the dimensions "Reconciliation", "future orientation", "forgiveness", or "forgetting" and similarly for the other five narrative frames.
Fourth, provide a narrative frame of your own that you observe and that has not been mentioned in the list above.
The format of the result to be provided is divided into 15 parts separated by a semicolon (;):
The following 6 ratings correspond to cases where the segment talks about the victims, thus if “victim presence” coding is 1 or 3, otherwise, the value is 0:
rating for *Citizenship*, followed by a semicolon (;),
rating for *Suffering*, followed by a semicolon (;),
rating for *Agency*, followed by a semicolon (;),
rating for *Reconciliation*, followed by a semicolon (;),
rating for *Victim Blaming*, followed by a semicolon (;),
rating for *Victims' Feelings Often Considered Unacceptable*, followed by a semicolon (;), 
the list of dimensions found that drive the above narrative frameworks in which the segment talks about victims, enclosed in square brackets “[”] and separated by commas (,), followed by a semicolon (;),

The following 6 ratings is for when the victims speak/testify themselves in the segment, thus if ‘victim presence’ coding is 2 or 3, otherwise the value is 0:
rating for *Citizenship*, followed by a semicolon (;),
rating for *Suffering*, followed by a semicolon (;),
rating for *Agency*, followed by a semicolon (;),
rating for *Reconciliation*, followed by a semicolon (;),
rating for *Victim Blaming*, followed by a semicolon (;),
rating for *Victims' Feelings Often Considered Unacceptable*, followed by a semicolon (;),
the list of dimensions found that drive the above narrative frameworks in which the victims themselves speak or testify, enclosed in square brackets “[”] and separated by commas (,), followed by a semicolon (;),

the additional narrative frame, followed by a semicolon (;),

and then a brief explanation in maximum 50 words. Do not use quotation marks."""

instruction_spanish_part2 = """Eres un científico social en el ámbito de la justicia transicional crítica, y tienes la tarea de clasificar extractos de los informes de diferentes comisiones de verdad y de esclarecimiento histórico, identificando cuando y donde los siguientes seis marcos narrativos están presentes:
*Ciudadanía*, este tema puede ser caracterizada por las siguientes dimensiones (¡no es necesario que estén todas presentes!): "titular(es) de derechos", "sujeto(s) de derechos", "obligaciones del Estado", "responsabilidades del Estado", "inclusión en el estado democrático", "democratización", "la (non)reconocimiento", "discriminación", "exclusión (social)", "marginalización", "desigualdad".
*Sufrimiento*, este tema puede ser caracterizada por las siguientes dimensiones (¡no es necesario que estén todas presentes!): "vulnerabilidad", "víctima(s) inocente(s)", "víctima(s) merecedora(s)", "víctima(s) heroica(s)", "víctima(s) pasiva(s)", "martir(es)", "violencia continua", "violencia cotidiana", "violencia histórica", "violencia estructural", "necesidad de apoyo", "sobrevictima".
*Capacidad de acción*, este tema puede ser caracterizada por las siguientes dimensiones (¡no es necesario que estén todas presentes!): "reclamación de los derechos humanos", "defensor de los derechos humanos", "resistencia (cotidiana)", "protesta", "lucha", "movilización (comunitaria)", "resiliencia", "resistencia", "sobreviviente", "grupo(s) de apoyo mutuo", "movimiento(s) de solidaridad", "protagonista", "líder", "organizador", "actor(as) de cambio", "conmemoración", "activista", "defensor de la memoria".
*Reconciliación*, este tema puede ser caracterizada por las siguientes dimensiones (¡no es necesario que estén todas presentes!): "reconciliación", "orientación al futuro", "perdón", "olvido".
*culpabilización de la víctima​*, Este tema puede identificarse cuando se discuten las acciones de las víctimas (como la protesta) justo antes o después de la mención de un acto violento, en particular cuando aparece la palabra ‘responsabilidad’ en relación con la persona o el grupo victimizados
*Sentimientos de las víctimas a menudo considerados inaceptables*. Este tema puede identificarse por referencias a sentimientos de las víctimas como "ira", "odio", "verguenza", "celos" o "(fantasías de) venganza" (¡no es necesario que estén todas presentes!).

Para este segmento, ya se ha determinado si el segmento habla explícitamente de las víctimas o si son las propias víctimas las que hablan o testifican utilizando la codificación: «1»: el segmento habla explicitamente de las víctimas; «2»: las víctimas hablan/testifican ello/as mismo/as; «3»: AMBOS, el segmento habla explitamente de las víctimas y las víctimas hablan/testifican ellas mismas en el segmento; «0»: no se menciona a las víctimas explicitamente. El código de «presencia de la víctima» para este segmento es {}.
En primer lugar, califique la presencia de cada uno de los seis marcos narrativos anteriores en el segmento con 0: no presente, 0.5: indirectamente presente, 1: fuertemente presente.
En segundo lugar, proporcione el razonamiento para llegar a los tres marcos más prevalentes en un máximo de 200 palabras.
En tercer lugar, proporcione una lista de las dimensiones que le han llevado a clasificar todos los marcos narrativos del segmento tal y como se han definido anteriormente, 1 lista para cuando el segmento habla de las víctimas y 1 lista para cuando las propias víctimas hablan en el segmento. Ten en cuenta que solo puedes elegir entre las dimensiones que se mencionan entre comillas (") después de cada uno de los seis marcos mencionados anteriormente. Por ejemplo, si clasificas un segmento como *Reconciliación*, solo puedes elegir entre las dimensiones "reconciliación", "orientación al futuro", "perdón" u "olvido", y lo mismo ocurre con los otros cinco marcos narrativos.
En cuarto lugar, identifique marcos narrativos adicionales utilizados para describir la experiencia de las víctimas de violencia.

El formato del resultado que se debe proporcionar se divide en 16 partes separadas por un punto y coma (;):

Las siguientes 6 calificaciones corresponden a los casos en que el segmento habla de las víctimas, por lo que si la codificación de «presencia de la víctima» es 1 o 3, el valor es 0; en caso contrario, el valor es 0:
calificación de *Ciudadanía*, seguido de un punto y coma (;),
calificación de *Sufrimiento*, seguido de un punto y coma (;),
calificación de *Capacidad de acción*, seguido de un punto y coma (;),
calificación de *Reconciliación*, seguido de un punto y coma (;),
calificación para «Culpar a las víctimas», seguido de un punto y coma (;),
calificación para *Los sentimientos de las víctimas a menudo se consideran inaceptables*, seguido de un punto y coma (;),
la lista de dimensiones encontradas que impulsan los marcos narrativos anteriores en los que el segmento habla de las víctimas, entre corchetes «[» »]» y separadas por comas (,), seguidas de un punto y coma (;), 

Las siguientes 6 calificaciones corresponden a los casos en que las víctimas hablan o testifican por sí mismas en el segmento, por lo que si la codificación de «presencia de la víctima» es 2 o 3, el valor es 0:
calificación para «ciudadanía», seguido de un punto y coma (;),
calificación de *Sufrimiento*, seguido de un punto y coma (;),
calificación para *Capacidad de acción*, seguido de un punto y coma (;),
calificación de *Reconciliación*, seguido de un punto y coma (;),
calificación para «Culpar a la víctima», seguido de un punto y coma (;),
calificación de *Sentimientos de las víctimas a menudo considerados inaceptables*, seguido de un punto y coma (;),
la lista de dimensiones encontradas que impulsan los marcos narrativos anteriores en los que las propias víctimas hablan o testifican, entre corchetes «[» »]» y separadas por comas (,), seguidas de un punto y coma (;) ,

el marco narrativo adicional, seguido de un punto y coma (;),

y luego una breve explicación de 50 palabras como máximo. No utilice comillas.

A continuación se muestran algunos ejemplos de cómo se codifican los marcos narrativos:

Segmento de ejemplo: La CEH ha formulado en el presente Informe las recomendaciones que ha juzgado necesarias para que en Guatemala se consolide la paz y la concordia nacional. Tomó en consideración los términos que en el apartado de “Finalidades” establecen el Acuerdo de Oslo y las peticiones formuladas en otros de los Acuerdos de Paz para que recomiende medidas de reparación a las víctimas, según se ha expresado anteriormente.
Codificación prevista para Ciudadanía: 0
Codificación prevista para Sufrimiento: 0
Codificación prevista para Capacidad de acción: 0
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: La formación de un Estado democrático se ha visto limitada permanentemente por esos condicionamientos de la historia. Su función ha consistido en mantener y conservar las estructuras de poder basadas en la explotación y la exclusión de los indígenas y de los mestizos empobrecidos. Fue necesario esperar los Acuerdos de Paz, concluidos en 1996, ciento setenta y cinco años después de la Independencia, para que el Estado guatemalteco pudiera perfilarse como un “Estado multiétnico, pluricultural y multilingüe” y, con ello, responder a una concepción de nación integradora, respetando la diversidad de sus ciudadanos.
Codificación prevista para Ciudadanía: 1
Codificación prevista para Sufrimiento: 0.5
Codificación prevista para Capacidad de acción: 0
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: El rasgo característico de este momento no siempre fue la muerte del opositor, sino la creación de un clima social de inseguridad generalizada. El efecto fue el miedo: miedo a ser denunciado, a perder el empleo, a no poder retornar al país, a participar en política, a organizarse para exigir derechos. En fin, a funcionar fuera del marco de una sociedad democrática de derecho, por la existencia de un régimen de excepción permanente.
Codificación prevista para Ciudadanía: 1
Codificación prevista para Sufrimiento: 1
Codificación prevista para Capacidad de acción: 0.5
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: Se calificaba así a los campesinos que en los años anteriores se habían organizado y que habían accedido a la tierra. En la concepción del nuevo régimen el agrarismo era sinónimo de comunismo, y los beneficiarios de la ley eran asumidos como comunistas. “ ...Al poco tiempo habíamos sembrado con mi papá, cuando comenzó a ponerse las cosas un poco feo ... decían que Arbenz no iba a dilatar, y ciertamente, pues la milpa estaba en elote cuando él cayó. La gente que estaba con el patrón nos acusaba que nosotros éramos unos comunistas ... entonces yo tuve que huir”.76 “ ...En la finca Caobanal en 1954, cuando entró Castillo Armas ... todos los que habían apoyado antes a Arbenz tuvieron que irse inmediatamente, porque mandaron a quemar las casas donde vivían con todo y los animales adentro ... con buena suerte se salvó mi familia”.
Codificación prevista para Ciudadanía: 0.5
Codificación prevista para Sufrimiento: 1
Codificación prevista para Capacidad de acción: 0.5
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: En un mensaje episcopal del 9 de mayo de 1967 los obispos guatemaltecos manifestaron su preocupación por la creciente ola de terror que sufría el país: “Cada día aumenta el número de huérfanos y viudas, son muchos los hombres arrancados violentamente de sus hogares por incógnitos secuestradores, son detenidos en lugares ignorados o violentamente asesinados apareciendo luego sus cadáveres horriblemente destrozados y profanados. Nos angustiamos con nuestro pueblo noble y pacífico, que, desde hace ya mucho tiempo, vive la zozobra, el temor y la angustia; la inseguridad se ahonda más y más”
Codificación prevista para Ciudadanía: 0.5
Codificación prevista para Sufrimiento: 1
Codificación prevista para Capacidad de acción: 0
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: Los efectos socioeconómicos y políticos provocados por el terremoto fueron importantes para la toma de conciencia de la población, constituyéndose en un importante elemento movilizador de la organización social, especialmente en las regiones mayas en donde se evidenciaron más las desigualdades existentes en el país, como lo expresa un dirigente campesino k’iche’: “Nosotros ya manejábamos unos elementos de la realidad nacional, pero el terremoto nos abrió los ojos, es decir, el terremoto vino a poner al desnudo nuestra realidad ... nos dábamos cuenta que la magnitud de la pobreza en el país era de tal modo que con láminas y casas prefabricadas no resolvíamos el asunto”.
Codificación prevista para Ciudadanía: 1
Codificación prevista para Sufrimiento: 1
Codificación prevista para Capacidad de acción: 1
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: En general, las relaciones entre la guerrilla y la población indígena tuvieron un carácter complejo y no unidireccional. Como ya se destacó en secciones precedentes de este capítulo, estas relaciones pusieron de manifiesto un interés compartido: la unión de los ladinos pobres con los mayas, los excluidos de siempre. Desde la perspectiva de la población maya hubo múltiples factores mediante los cuales se expresó este interés, como revelan las siguientes opiniones.
Codificación prevista para Ciudadanía: 1
Codificación prevista para Sufrimiento: 1
Codificación prevista para Capacidad de acción: 0,5
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: Paralelamente a los intentos de reactivación económica, el Gobierno emprendió una brutal campaña represiva contra el movimiento social, tanto en el área rural como en la urbana. El asesinato y la desaparición sistemática de líderes renombrados, así como las masacres de campesinos en el interior del país tuvo fuerte repercusión en el ámbito internacional. Guatemala se convirtió en un objeto de frecuentes sanciones y del aislamiento internacional por la intensidad de la represión estatal.
Codificación prevista para Ciudadanía: 0
Codificación prevista para Sufrimiento: 0
Codificación prevista para Capacidad de acción: 0
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: El movimiento social que había alcanzado un gran desarrollo en los primeros dos años del Gobierno de Laugerud García había sufrido importantes golpes por las acciones represivas que se endurecieron después del terremoto de 1976. Esto derivó en una politización y radicalización de las organizaciones sociales, tanto urbanas como rurales, al compás de su gradual movilización. Este proceso de radicalización fue resultado de un cúmulo de factores donde influyeron, además de la violencia represiva, la continuación de las exclusiones sociales y políticas, el trabajo ideológico de la guerrilla y la influencia externa de los éxitos del Frente Sandinista de Liberación Nacional (FSLN), en Nicaragua, y del Frente Farabundo Martí para la Liberación Nacional (FMLN), en El Salvador.
Codificación prevista para Ciudadanía: 1
Codificación prevista para Sufrimiento: 0,5
Codificación prevista para Capacidad de acción: 1
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0,5
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0,5

Segmento de ejemplo: En este marco, el Ejército percibía la participación indígena ya sea en el movimiento social o insurgente como producto de su falta de integración al Estado y de un débil sentido nacionalista. A su vez, en buena medida alimentado por preceptos racistas, argumentaba que por inmadurez los indígenas eran fácilmente manipulables por la acción política de la guerrilla. El Ejército consideraba que la participación indígena y campesina estaba determinada por la presión y amenaza guerrillera, así como por el resentimiento y el abandono en que se encontraban. En consecuencia, definían su conexión con los insurgentes como potencialmente peligrosa.330 Según explicó Francisco Bianchi, portavoz de Ríos Montt en 1982, en una controvertida entrevista: “Los guerrilleros conquistaron muchos colaboradores indígenas, entonces los indígenas eran subversivos, no. ¿Y, como se lucha en contra de la insurgencia? Netamente, tendría que matar a los indígenas porque ellos estaban colaborando con la subversión”.
Codificación prevista para Ciudadanía: 1
Codificación prevista para Sufrimiento: 1
Codificación prevista para Capacidad de acción: 0,5
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: bana y rural. Todo ello explica entonces el interés de los militares en los siguientes años por darle a la política contrainsurgente un carácter de esfuerzo estatal y por institucionalizar muchos de sus procedimientos. Como indicó el mismo Ríos Montt: “Naturalmente, si una operación subversiva existe donde los indígenas están involucrados con la guerrilla, los indígenas morirán. Sin embargo no es la filosofía del Ejército matar indígenas, sino reconquistarlos y ayudarlos”.
Codificación prevista para Ciudadanía: 0,5
Codificación prevista para Sufrimiento: 1
Codificación prevista para Capacidad de acción: 0
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: En el mismo período surgió el Grupo de Apoyo Mutuo por el Aparecimiento de Familiares Desaparecidos (GAM),346 organización que jugó un importante papel en el marco de silencio que existía en el país sobre víctimas del enfrentamiento. De esta forma los planteamientos y demandas por las reivindicaciones laborales de corte tradicional y otras vinculadas a los efectos sociales del enfrentamiento pasaron a primer plano. En los primeros meses de 1985 se produjeron protestas de los universitarios, el GAM, el magisterio, sindicatos estatales y organizaciones de apoyo a los consumidores. En ese año los sindicatos volvieron a conmemorar el 1o. de mayo, por primera vez desde 1980 cuando se habían producido saldos fatales.
Codificación prevista para Ciudadanía: 0.5
Codificación prevista para Sufrimiento: 0
Codificación prevista para Capacidad de acción: 1
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: Los cambios que se produjeron en la cúpula del Ejército entre junio y septiembre fueron interpretados como expresiones de problema internos, de descontentos hacia las acciones gubernativas o como medidas previsoras de los altos mandos para evitar nuevos movimientos golpistas y mantener la línea de la estabilidad nacional. En junio, fue asesinado en Petén el estadounidense Michael Devine, por lo que la Embajada de su país presionó por que se investigara el hecho. Asimismo, en septiembre y octubre fueron asesinados la antropóloga Myrna Mack,388 cuyo trabajo de investigación se centraba en las condiciones de la población desplazada, y el político Humberto González Gamarra, dirigente socialdemócrata que buscaba ampliar espacios de participación política para sectores de izquierda.
Codificación prevista para Ciudadanía: 0.5
Codificación prevista para Sufrimiento: 1
Codificación prevista para Capacidad de acción: 1
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0

Segmento de ejemplo: El proceso de reactivación del movimiento social en el campo se dio con características similares, aunque no igualmente movilizadoras. El CUC reapareció luego de varios años en la clandestinidad al ser considerado un brazo político de la guerrilla. Más que apoyar la discusión de la tenencia de la tierra, que otros grupos planteaban, buscaba la movilización de los trabajadores agrícolas cañeros y cafetaleros en pro de mejores condiciones de trabajo, en especial en relación con el salario mínimo, además de incluir las demandas de los sectores campesinos afectados por el enfrentamiento.401 De esta forma inició tanto una lucha reivindicativa como de legitimación de su presencia política. Sus demandas crearon tensión con el Gobierno y la UNAGRO, sin que se produjeran resultados concretos, pues se concentraron en mantener la discusión en un órgano de concertación oficial y técnico sobre la posibilidad o no de incrementar salarios.
Codificación prevista para Ciudadanía: 1
Codificación prevista para Sufrimiento: 0
Codificación prevista para Capacidad de acción: 1
Codificación prevista para Reconciliación: 0
Codificación prevista para Culpar a las víctimas: 0
Codificación prevista para Los sentimientos de las víctimas a menudo se consideran inaceptables: 0
"""

# Call LLM API

In [126]:
import time

def analyze_message(text, instruction, model = 'gpt-4', temperature=1):
    print(f"Analyzing message...")
    
    response = None
    tries = 0
    failed = True
    
    while(failed):
        try:
            response = client.chat.completions.create(
                model = model, 
                temperature=temperature,
                messages=[
                        {"role": "system", "content": f"'{instruction}'"}, #The system instruction tells the bot how it is supposed to behave
                        {"role": "user", "content": f"'{text}'"} #This provides the text to be analyzed.
                    ]
            )
            failed = False

        #Handle errors.
        # If the API gets an error, perhaps because it is overwhelmed, we wait 10 seconds and then we try again. 
        # We do this 10 times, and then we give up.
        except openai.APIError as e:
            print(f"OpenAI API returned an API Error: {e}")
            
            if tries < 10:
                print(f"Caught an APIError: {e}. Waiting 10 seconds and then trying again...")
                failed = True
                tries += 1
                time.sleep(10)
            else:
                print(f"Caught an APIError: {e}. Too many exceptions. Giving up.")
                raise e
                
        except openai.ServiceUnavailableError as e:
            print(f"OpenAI API returned an ServiceUnavailable Error: {e}")
            
            if tries < 10:
                print(f"Caught a ServiceUnavailable error: {e}. Waiting 10 seconds and then trying again...")
                failed = True
                tries += 1
                time.sleep(10)
            else:
                print(f"Caught a ServiceUnavailable error: {e}. Too many exceptions. Giving up.")
                raise e
            
        except openai.APIConnectionError as e:
            print(f"Failed to connect to OpenAI API: {e}")
            pass
        except openai.RateLimitError as e:
            print(f"OpenAI API request exceeded rate limit: {e}")
            pass
        
        #If the text is too long, we truncate it and try again. Note that if you get this error, you probably want to chunk your texts.
        except openai.InvalidRequestError as e:
            #Shorten request text
            print(f"Received a InvalidRequestError. Request likely too long; cutting 10% of the text and trying again. {e}")
            time.sleep(5)
            words = text.split()
            num_words_to_remove = round(len(words) * 0.1)
            remaining_words = words[:-num_words_to_remove]
            text = ' '.join(remaining_words)
            failed = True
        
        except Exception as e:
            print(f"Caught unhandled error.")
            pass
            
    result = ''
    for choice in response.choices:
        result += choice.message.content
    
    return result 

# Extract text and detect text properties

In [35]:
import fitz
import pandas as pd

folder_path = './01. Corpus/02. Guatemala'   # no trailing slash needed
pdf_files = glob.glob(os.path.join(folder_path, '*.pdf'))

for file_path in pdf_files:
    filename = os.path.basename(file_path)
    base, _ = os.path.splitext(filename)  # remove .pdf

    try:
        # --- Extract with textract ---
        doc = fitz.open(file_path)
    except Exception as e:
        errors.append((filename, str(e)))
        print(f"[ERROR] {filename}: {e}")

In [56]:
segments = []

for page_num, page in enumerate(doc, start=1):
    page_dict = page.get_text("dict")

    for block in page_dict.get("blocks", []):
        if "lines" not in block:
            continue

        block_lines = []
        font_sizes = []
        current_segment_lines = []

        for line in block["lines"]:
            line_text = []
            line_origin_x = None

            for span in line.get("spans", []):
                text = span.get("text", "")
                origin = span.get("origin", None)

                if not text.strip():
                    continue

                if line_origin_x is None and origin:
                    line_origin_x = origin[0]

                line_text.append(text)
                font_sizes.append(span["size"])

            if not line_text:
                continue

            joined_line = "".join(line_text).strip()

            if line_origin_x is not None:
                rounded_x = round(line_origin_x)
                if rounded_x in {96, 114}:
                    # save previous segment if it exists
                    if current_segment_lines:
                        paragraph_text = " ".join(current_segment_lines).strip()
                        if paragraph_text:
                            segment_font_size = round(float(pd.Series(font_sizes).median()), 2)

                            if not (
                                segment_font_size <= 9.25
                                or segment_font_size > 10.5
                                or segment_font_size == 9.77
                                or len(paragraph_text) <= 10
                            ):
                                segments.append(
                                    {
                                        "filename": filename,
                                        "page": page_num,
                                        "segment": paragraph_text,
                                        "font_size": segment_font_size,
                                    }
                                )

                    # start a new segment
                    current_segment_lines = [joined_line]
                    continue

            # otherwise, keep accumulating
            current_segment_lines.append(joined_line)

        # finalize last segment in block
        if current_segment_lines:
            paragraph_text = " ".join(current_segment_lines).strip()
            if paragraph_text and font_sizes:
                segment_font_size = round(float(pd.Series(font_sizes).median()), 2)

                if not (
                    segment_font_size <= 9.25
                    or segment_font_size > 10.5
                    or segment_font_size == 9.77
                    or len(paragraph_text) <= 10
                ):
                    segments.append(
                        {
                            "filename": filename,
                            "page": page_num,
                            "segment": paragraph_text,
                            "font_size": segment_font_size,
                        }
                    )

In [58]:
# Merge logic: join consecutive segments 
merged_segments = []
i = 0

while i < len(segments):
    cur = segments[i]

    if i + 1 < len(segments):
        nxt = segments[i + 1]

        text1 = cur["segment"].rstrip()
        text2 = nxt["segment"].lstrip()

        # condition 1: first does NOT end with '.'
        cond1 = not text1.endswith(".") and not text1.endswith(":")

        # condition 2: second does NOT start a new sentence
        # lowercase first letter OR starts with four digits
        starts_with_four_digits = len(text2) >= 4 and text2[:4].isdigit()

        first_alpha = ""
        for ch in text2:
            if ch.isalpha():
                first_alpha = ch
                break

        starts_with_lowercase = first_alpha != "" and first_alpha.islower()

        cond2 = starts_with_lowercase or starts_with_four_digits

        if cond1 and cond2:
            # merge the two segments
            merged_text = text1 + " " + text2
            merged_font_size = max(cur["font_size"], nxt["font_size"])

            merged_segments.append(
                {
                    "filename": filename,
                    "page": cur["page"],
                    "segment": merged_text,
                    # "has_italic": merged_has_italic,
                    # "has_bold": merged_has_bold,
                    "font_size": merged_font_size,
                }
            )
            i += 2
            continue

    # default: keep current as-is
    merged_segments.append(cur)
    i += 1

In [59]:
segments_with_ids = []
for sid, seg in enumerate(merged_segments, start=1):
    seg = dict(seg)
    seg["segment_id"] = sid
    segments_with_ids.append(seg)

df_segments = pd.DataFrame(
    segments_with_ids,
    columns=[
        "filename",
        "page",
        "segment_id",
        "segment",
    ],
)

df_segments["victim_presence"] = pd.NA
df_segments["reasoning_victim_presence"] = pd.NA

output_file = "df_segments.xlsx"
df_segments.to_excel(output_file, index=False)

# Batch analysis in a loop (batch 1)

In [63]:
import json
import time
import pandas as pd
from openai import OpenAI

# 1. Prepare base dataframe with only rows needing classification
df_segments_temp = df_segments[df_segments["victim_presence"].isna()].copy()
df_segments_temp["segment_id"] = df_segments_temp["segment_id"].astype(int)

# Optional: persist snapshot
df_segments_temp.to_pickle("data.pkl")

# 2. Helper functions 
def prepare_batch_file(output_file, data_file, instruction):
    with open(output_file, "w", encoding="utf-8") as new_file:
        with open(data_file, "r", encoding="utf-8") as file:
            for line in file:
                json_object = json.loads(line)
                payload = {
                    "custom_id": f"{json_object['segment_id']}",
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": {
                        "model": MODEL,
                        "messages": [
                            {"role": "system", "content": instruction},
                            {"role": "user", "content": json_object["segment"]},
                        ],
                        "max_completion_tokens": 3000,
                    },
                }
                new_file.write(json.dumps(payload, ensure_ascii=False) + "\n")

def parse_result(result: str):
    # strip stray quotes
    result = (result or "").strip("'\"")
    # expected "number;reason"
    return result.split(";", 1)

# ensure result columns exist
output_columns = ["victim_presence", "reasoning_victim_presence"]
for col in output_columns:
    if col not in df_segments_temp.columns:
        df_segments_temp[col] = pd.NA

In [ ]:
# 3. Loop over df_segments_temp in chunks of 1000
chunk_size = 1000
failed_segments = []

for batch_idx, start in enumerate(range(1000, len(df_segments_temp), chunk_size), start=1):
    end = start + chunk_size
    df_chunk = df_segments_temp.iloc[start:end].copy()
    if df_chunk.empty:
        break

    print(f"\n=== Processing batch {batch_idx} (rows {start}:{end}) ===")

    # 3a. Create input JSONL for this chunk
    input_file_path = f"df_segments_batch{batch_idx}.jsonl"

    with open(input_file_path, "w", encoding="utf-8") as f:
        for _, row in df_chunk.iterrows():
            entry = {
                "segment_id": int(row["segment_id"]),
                "segment": row["segment"],  # change to row["segment_text"] if needed
            }
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    # 3b. Prepare batch request file
    input_batch_file = f"df_segments_batch{batch_idx}_classifications.jsonl"
    prepare_batch_file(input_batch_file, input_file_path, instruction_spanish_part1)

    # 3c. Upload batch file & create batch job
    batch_input_file = client.files.create(
        file=open(input_batch_file, "rb"),
        purpose="batch",
    )

    batch = client.batches.create(
        input_file_id=batch_input_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={"description": f"Victim presence classification batch {batch_idx}"},
    )
    print("Batch ID:", batch.id)

    batch_id = batch.id

    # 3d. Poll until batch completes
    while True:
        status = client.batches.retrieve(batch_id).status
        print(f"Current status (batch {batch_idx}):", status)

        if status in ["completed", "failed", "expired", "cancelling", "cancelled"]:
            break

        time.sleep(10)

    batch_properties = client.batches.retrieve(batch_id)
    batch_outputfile_id = batch_properties.output_file_id
    print("Batch properties:", batch_properties)
    print("Batch output file ID:", batch_outputfile_id)

    # 3e. Download output & parse
    file_response = client.files.content(batch_outputfile_id)

    # Save raw output if you want
    raw_output_path = f"batch{batch_idx}_output_raw.jsonl"
    with open(raw_output_path, "w", encoding="utf-8") as f:
        f.write(file_response.text)

    for line in file_response.text.splitlines():
        if not line.strip():
            continue
        obj = json.loads(line)
        segment_id = int(obj.get("custom_id"))

        content = (
            obj.get("response", {})
              .get("body", {})
              .get("choices", [{}])[0]
              .get("message", {})
              .get("content", None)
        )

        try:
            victim_presence, reasoning_victim_presence = parse_result(content)
        except ValueError as e:
            print(f"[Warning] parse_result failed for segment_id {segment_id}: {e}")
            print(line)
            failed_segments.append(segment_id)
            continue

        df_segments_temp.loc[
            df_segments_temp["segment_id"] == segment_id, "victim_presence"
        ] = victim_presence

        df_segments_temp.loc[
            df_segments_temp["segment_id"] == segment_id, "reasoning_victim_presence"
        ] = reasoning_victim_presence

# 4. Done: df_segments_temp now has updated classifications
print("All batch 1 segments processed.")
print("Failed segment_ids:", failed_segments)

## Corrections batch 1

## Create set with all the to be classified segments after batch 1

In [89]:
# Align on segment_id and overwrite values from df_segment_temp

df_segments = df_segments.set_index("segment_id")
df_segments_temp = df_segments_temp.set_index("segment_id")

df_segments.loc[df_segments_temp.index,['victim_presence','reasoning_victim_presence']] = df_segments_temp[['victim_presence','reasoning_victim_presence']]

df_segments.reset_index(inplace=True)
df_segments_temp.reset_index(inplace=True)

In [85]:
df_segments["victim_presence"] = (
    pd.to_numeric(df_segments["victim_presence"], errors="coerce")
    .astype("Int64")
)

# Batch analysis in a loop (batch 2)

In [90]:
# 1. Base filter: only rows with victim_presence in {1,2,3} 
df_segments_temp2 = df_segments[df_segments['victim_presence'].isin([1, 2, 3])].copy()

# Lookup for victim_presence (used to fill the instruction)
vp_lookup = dict(df_segments_temp2[['segment_id', 'victim_presence']].values)

# Ensure result columns exist
output_columns2 = [
    'narrative_frame_citizenship_passive','narrative_frame_suffering_passive',
    'narrative_frame_agency_passive','narrative_frame_reconciliation_passive',
    'narrative_frame_victimblaming_passive','narrative_frame_angerhatred_passive',
    'narrative_frame_passive_dimensions',
    'narrative_frame_citizenship_active','narrative_frame_suffering_active',
    'narrative_frame_agency_active','narrative_frame_reconciliation_active',
    'narrative_frame_victimblaming_active','narrative_frame_angerhatred_active',
    'narrative_frame_active_dimensions',
    'narrative_frame_additional','reasoning_narrative_frame'
]

for col in output_columns2:
    if col not in df_segments_temp2.columns:
        df_segments_temp2[col] = pd.NA  # or None

# (Optional) persist snapshot
df_segments_temp2.to_pickle("data_batch2.pkl")

#  2. Helper functions
def prepare_batch2_file(output_file, data_file, instruction_template):
    with open(output_file, 'w', encoding="utf-8") as new_file:
        with open(data_file, 'r', encoding="utf-8") as file:
            for _, line in enumerate(file):
                json_object = json.loads(line)
                sid = json_object['segment_id']
                victim_presence = vp_lookup.get(sid)
                instruction_filled = instruction_template.format(victim_presence)
                payload = {
                    "custom_id": f"{sid}",
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": {
                        "model": MODEL,
                        "messages": [
                            {"role": "system", "content": instruction_filled},
                            {"role": "user", "content": json_object["segment"]},
                        ],
                        "max_completion_tokens": 10000,
                    },
                }
                new_file.write(json.dumps(payload, ensure_ascii=False) + '\n')

def parse_result2(result: str):
    # strip surrounding quotes, if any
    result = (result or "").strip("'\"")
    # expect 16 fields separated by ';'
    return result.split(';', 15)

In [ ]:


# 3. Loop over df_segments_temp in chunks of 1000
failed_segments = []
chunk_size = 1000

for batch_idx, start in enumerate(range(0, len(df_segments_temp2), chunk_size), start=1):
    end = start + chunk_size
    df_chunk = df_segments_temp2.iloc[start:end].copy()
    if df_chunk.empty:
        break

    print(f"\n=== Processing batch 2 - chunk {batch_idx} (rows {start}:{end}) ===")

    # 3.a Create jsonl input file for this chunk 
    input_file_path2 = f'df_segments_batch2_{batch_idx}.jsonl'

    with open(input_file_path2, "w", encoding="utf-8") as f:
        for _, row in df_chunk.iterrows():
            entry = {
                "segment_id": row["segment_id"],
                "segment": row["segment"],
                "victim_presence": row["victim_presence"],
            }
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    # 3.b Prepare batch2 request file for this chunk 
    input_batch2_file = f'df_segments_batch2_{batch_idx}_classifications.jsonl'
    data_file2 = input_file_path2

    prepare_batch2_file(input_batch2_file, data_file2, instruction_spanish_part2)

    # 3.c Upload and create batch 
    batch2_input_file = client.files.create(
        file=open(input_batch2_file, "rb"),
        purpose="batch",
    )

    batch2_input = client.batches.create(
        input_file_id=batch2_input_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": f"Narrative frames classification - chunk {batch_idx}",
        },
    )
    print("Batch ID:", batch2_input.id)

    batch2_input_id = batch2_input.id

    # 3.d Poll until done 
    while True:
        status = client.batches.retrieve(batch2_input_id).status
        print("Current status for", batch2_input_id, ":", status)

        if status in ["completed", "failed", "expired", "cancelling", "cancelled"]:
            break

        time.sleep(10)

    batch2_properties = client.batches.retrieve(batch2_input_id)
    batch2_outputfile_id = batch2_properties.output_file_id
    print("Batch properties: ", batch2_properties)
    print("Batch output file: ", batch2_outputfile_id)

    # 3.e Download and save raw output
    file_response_batch2 = client.files.content(batch2_outputfile_id)

    raw_output_path = f'test_batch2_output_chunk{batch_idx}.jsonl'
    with open(raw_output_path, "w", encoding="utf-8") as f:
        f.write(json.dumps(file_response_batch2.text, ensure_ascii=False))

    # 3.f Parse results back into df_segments_temp2 
    for line in file_response_batch2.text.splitlines():
        if not line.strip():
            continue

        obj = json.loads(line)
        segment_id = int(obj.get("custom_id"))
        content = (
            obj.get("response", {})
              .get("body", {})
              .get("choices", [{}])[0]
              .get("message", {})
              .get("content", None)
        )

        try:
            (
                narrative_frame_citizenship_passive,
                narrative_frame_suffering_passive,
                narrative_frame_agency_passive,
                narrative_frame_reconciliation_passive,
                narrative_frame_victimblaming_passive,
                narrative_frame_angerhatred_passive,
                narrative_frame_passive_dimensions,
                narrative_frame_citizenship_active,
                narrative_frame_suffering_active,
                narrative_frame_agency_active,
                narrative_frame_reconciliation_active,
                narrative_frame_victimblaming_active,
                narrative_frame_angerhatred_active,
                narrative_frame_active_dimensions,
                narrative_frame_additional,
                reasoning_narrative_frame,
            ) = parse_result2(content)
        except ValueError as e:
            print(f"[Warning] parse_result2 failed for segment_id {segment_id}: {e}")
            print(line)
            failed_segments.append(segment_id)
            continue

        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_citizenship_passive'] = narrative_frame_citizenship_passive
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_suffering_passive'] = narrative_frame_suffering_passive
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_agency_passive'] = narrative_frame_agency_passive
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_reconciliation_passive'] = narrative_frame_reconciliation_passive
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_victimblaming_passive'] = narrative_frame_victimblaming_passive
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_angerhatred_passive'] = narrative_frame_angerhatred_passive
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_passive_dimensions'] = narrative_frame_passive_dimensions

        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_citizenship_active'] = narrative_frame_citizenship_active
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_suffering_active'] = narrative_frame_suffering_active
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_agency_active'] = narrative_frame_agency_active
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_reconciliation_active'] = narrative_frame_reconciliation_active
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_victimblaming_active'] = narrative_frame_victimblaming_active
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_angerhatred_active'] = narrative_frame_angerhatred_active
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_active_dimensions'] = narrative_frame_active_dimensions

        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'narrative_frame_additional'] = narrative_frame_additional
        df_segments_temp2.loc[df_segments_temp2['segment_id'] == segment_id, 'reasoning_narrative_frame'] = reasoning_narrative_frame

print("All batch 2 segments processed.")
print("Failed segment_ids:", failed_segments)
low_beep()

## Corrections batch 2

In [171]:
df_segments_temp2["narrative_frame_passive_dimensions"] = (
    df_segments_temp2["narrative_frame_passive_dimensions"]
    .astype(str)
    .str.replace('"', '', regex=False)
    .str.replace('“', '', regex=False)
    .str.replace('”', '', regex=False)
)

df_segments_temp2["narrative_frame_active_dimensions"] = (
    df_segments_temp2["narrative_frame_active_dimensions"]
    .astype(str)
    .str.replace('"', '', regex=False)
    .str.replace('“', '', regex=False)
    .str.replace('”', '', regex=False)
)

In [173]:
df_segments_temp2 = df_segments_temp2.replace(' 0,5', '0.5')
df_segments_temp2 = df_segments_temp2.replace('0,5', '0.5')
df_segments_temp2 = df_segments_temp2.replace('\n0,5', '0.5')


## Create set with all the to be classified segments after batch 2

In [174]:
# Align on segment_id and overwrite values from df_segment_temp2
for col in frame_cols:
    if col not in df_segments.columns:
        df_segments[col] = pd.NA  # or None

df_segments = df_segments.set_index("segment_id")
df_segments_temp2 = df_segments_temp2.set_index("segment_id")

df_segments.loc[df_segments_temp2.index,frame_cols] = df_segments_temp2[frame_cols]

df_segments.reset_index(inplace=True)
df_segments_temp2.reset_index(inplace=True)


Create df_segment_analysis dataframe to conduct further analysis

Expand on the dimension columns narrative_frame_passive_dimensions and narrative_frame_active_dimensions

In [14]:
df_segments_analysis = df_segments

In [ ]:
import ast
import re

dimension_cols = ["narrative_frame_passive_dimensions",
        "narrative_frame_active_dimensions"]

unique_dims = set()

def parse_dim_list(val):
    if not isinstance(val, str):
        return []

    # Try Python literal parsing
    try:
        parsed = ast.literal_eval(val)
    except:
        parsed = None

    # If it parsed to a list / tuple / set
    if isinstance(parsed, (list, tuple, set)):
        return [str(x).strip() for x in parsed]

    # If it parsed to a single value (int, str, etc.)
    if parsed is not None:
        return [str(parsed).strip()]

    # Fallback: handle [a, b, c]
    s = val.strip()
    if s.startswith("[") and s.endswith("]"):
        s = s[1:-1]

    return [x.strip() for x in s.split(",") if x.strip()]

for col in dimension_cols:
    for val in df_segments_analysis[col].dropna():
        items = parse_dim_list(val)
        for x in items:
            unique_dims.add(x.strip())

# Convert back to normal list
unique_dims = sorted(unique_dims)

print(unique_dims)

print(len(unique_dims))

In [ ]:

frame_cols = ['narrative_frame_citizenship_passive','narrative_frame_suffering_passive','narrative_frame_agency_passive','narrative_frame_reconciliation_passive','narrative_frame_victimblaming_passive','narrative_frame_angerhatred_passive',
 'narrative_frame_citizenship_active','narrative_frame_suffering_active','narrative_frame_agency_active','narrative_frame_reconciliation_active','narrative_frame_victimblaming_active','narrative_frame_angerhatred_active']

df_segments_analysis["victim_presence"] = df_segments_analysis["victim_presence"].astype(float)
df_segments_analysis[frame_cols] = df_segments_analysis[frame_cols].fillna(0)
df_segments_analysis = df_segments_analysis.replace('0,5', '0.5')
df_segments_analysis = df_segments_analysis.replace('\n0,5', '0.5')

df_segments_analysis[frame_cols] = df_segments_analysis[frame_cols].astype(float)

df_segments_analysis["narrative_frame_citizenship"] = (
    df_segments_analysis[["narrative_frame_citizenship_passive",
                          "narrative_frame_citizenship_active"]]
    .max(axis=1)
)
df_segments_analysis["narrative_frame_suffering"] = (
    df_segments_analysis[["narrative_frame_suffering_passive",
                          "narrative_frame_suffering_active"]]
    .max(axis=1)
)
df_segments_analysis["narrative_frame_agency"] = (
    df_segments_analysis[["narrative_frame_agency_passive",
                          "narrative_frame_agency_active"]]
    .max(axis=1)
)
df_segments_analysis["narrative_frame_reconciliation"] = (
    df_segments_analysis[["narrative_frame_reconciliation_passive",
                          "narrative_frame_reconciliation_active"]]
    .max(axis=1)
)
df_segments_analysis["narrative_frame_victimblaming"] = (
    df_segments_analysis[["narrative_frame_victimblaming_passive",
                          "narrative_frame_victimblaming_active"]]
    .max(axis=1)
)
df_segments_analysis["narrative_frame_angerhatred"] = (
    df_segments_analysis[["narrative_frame_angerhatred_passive",
                          "narrative_frame_angerhatred_active"]]
    .max(axis=1)
)

In [ ]:
import ast
import re
import math

# Columns containing dimension lists
passive_col = "narrative_frame_passive_dimensions"
active_col  = "narrative_frame_active_dimensions"

def parse_list(x):
    """
    Parse values like "[a,b,c]" or "a,b,c" into ['a','b','c'].
    Returns [] for NaN or malformed rows.
    """
    # Already a list
    if isinstance(x, list):
        return x

    # NaN (pandas float NaN)
    if isinstance(x, float) and math.isnan(x):
        return []

    # Strings: "[a,b,c]" or "a,b,c"
    if isinstance(x, str):
        s = x.strip()
        # remove surrounding brackets if present
        if s.startswith('[') and s.endswith(']'):
            s = s[1:-1].strip()
        if not s:
            return []
        return [item.strip() for item in s.split(',')]

    # Anything else
    return []

# Parse both columns into real lists
df_segments_analysis["passive_list"] = df_segments_analysis[passive_col].apply(parse_list)
df_segments_analysis["active_list"]  = df_segments_analysis[active_col].apply(parse_list)


# Clean phrases → valid column names (underscores)

def clean_name(s):
    # Lowercase, replace non-alphanumeric with underscores
    s = s.lower()
    s = re.sub(r"[^0-9a-zA-Z]+", "_", s)
    return s.strip("_")

# Create one columns for passive + active

for dim in unique_dims:
    col_passive = f"dimension_passive_{clean_name(dim)}"
    col_active  = f"dimension_active_{clean_name(dim)}"

    df_segments_analysis[col_passive] = df_segments_analysis["narrative_frame_passive_dimensions"].apply(
        lambda lst: 1 if dim in lst else 0
    )
    df_segments_analysis[col_active]  = df_segments_analysis["narrative_frame_active_dimensions"].apply(
        lambda lst: 1 if dim in lst else 0
    )

print("Created columns:")
print([f"dimension_passive_{clean_name(d)}" for d in unique_dims])
print([f"dimension_active_{clean_name(d)}" for d in unique_dims])

In [208]:
output_file1 = "df_segments_xxx.parquet"
df_segments.to_parquet(output_file1, engine="pyarrow", index=False)

output_file2 = "df_segments_temp_xxx.parquet"
df_segments_temp.to_parquet(output_file2, engine="pyarrow", index=False)

output_file3 = "df_segments_temp2_xxx.parquet"
df_segments_temp2.to_parquet(output_file3, engine="pyarrow", index=False)

output_file4 = "df_segments_analysis_xxx.parquet"
df_segments_analysis.to_parquet(output_file4, engine="pyarrow", index=False)

# Narrative Frame Dimension Analysis

In [20]:
import pandas as pd

data = [
    ("Ciudadanía", "Titular(es) de derechos"),
    ("Ciudadanía", "Sujeto(s) de derechos"),
    ("Ciudadanía", "Obligaciones del Estado"),
    ("Ciudadanía", "Responsabilidades del Estado"),
    ("Ciudadanía", "Inclusión en el estado democrático"),
    ("Ciudadanía", "Democratización"),
    ("Ciudadanía", "La (non)reconocimiento"),
    ("Ciudadanía", "Discriminación"),
    ("Ciudadanía", "Exclusión (social)"),
    ("Ciudadanía", "Marginalización"),
    ("Ciudadanía", "Desigualdad"),

    ("Sufrimiento", "Vulnerabilidad"),
    ("Sufrimiento", "Víctima(s) inocente(s)"),
    ("Sufrimiento", "Víctima(s) merecedora(s)"),
    ("Sufrimiento", "Víctima(s) heroica(s)"),
    ("Sufrimiento", "Víctima(s) pasiva(s)"),
    ("Sufrimiento", "Martir(es)"),
    ("Sufrimiento", "Violencia continua"),
    ("Sufrimiento", "Violencia cotidiana"),
    ("Sufrimiento", "Violencia histórica"),
    ("Sufrimiento", "Violencia estructural"),
    ("Sufrimiento", "Necesidad de apoyo"),
    ("Sufrimiento", "Sobrevictima"),

    ("Capacidad de acción", "Reclamación de los derechos humanos"),
    ("Capacidad de acción", "Defensor de los derechos humanos"),
    ("Capacidad de acción", "Resistencia (cotidiana)"),
    ("Capacidad de acción", "Protesta"),
    ("Capacidad de acción", "Lucha"),
    ("Capacidad de acción", "Movilización (comunitaria)"),
    ("Capacidad de acción", "Resiliencia"),
    ("Capacidad de acción", "Resistencia"),
    ("Capacidad de acción", "Sobreviviente"),
    ("Capacidad de acción", "Grupo(s) de apoyo mutuo"),
    ("Capacidad de acción", "Movimiento(s) de solidaridad"),
    ("Capacidad de acción", "Protagonista"),
    ("Capacidad de acción", "Líder"),
    ("Capacidad de acción", "Organizador"),
    ("Capacidad de acción", "Actor(as) de cambio"),
    ("Capacidad de acción", "Conmemoración"),
    ("Capacidad de acción", "Activista"),
    ("Capacidad de acción", "Defensor de la memoria"),

    ("Reconciliación", "Reconciliación"),
    ("Reconciliación", "Orientación al futuro"),
    ("Reconciliación", "Perdón"),
    ("Reconciliación", "Olvido"),

    ("Sentimientos de las víctimas a menudo considerados inaceptables", "Ira"),
    ("Sentimientos de las víctimas a menudo considerados inaceptables", "Odio"),
    ("Sentimientos de las víctimas a menudo considerados inaceptables", "Verguenza"),
    ("Sentimientos de las víctimas a menudo considerados inaceptables", "(Fantasías de) venganza")
]

df_narrative_frame_dimension = pd.DataFrame(data, columns=["Narrative Frame", "Dimension"])

frame_to_passive_col = {
    "Ciudadanía": "narrative_frame_citizenship_passive",
    "Sufrimiento": "narrative_frame_suffering_passive",
    "Capacidad de acción": "narrative_frame_agency_passive",
    "Reconciliación": "narrative_frame_reconciliation_passive",
    "Sentimientos de las víctimas a menudo considerados inaceptables":
        "narrative_frame_angerhatred_passive"
}

df_narrative_frame_dimension["Column name narrative frame passive"] = (
    df_narrative_frame_dimension["Narrative Frame"]
    .map(frame_to_passive_col)
)

frame_to_active_col = {
    "Ciudadanía": "narrative_frame_citizenship_active",
    "Sufrimiento": "narrative_frame_suffering_active",
    "Capacidad de acción": "narrative_frame_agency_active",
    "Reconciliación": "narrative_frame_reconciliation_active",
    "Sentimientos de las víctimas a menudo considerados inaceptables":
        "narrative_frame_angerhatred_active"
}

df_narrative_frame_dimension["Column name narrative frame active"] = (
    df_narrative_frame_dimension["Narrative Frame"]
    .map(frame_to_active_col)
)

df_narrative_frame_dimension['Column name dimension passive'] = (
    "dimension_passive_" +
    df_narrative_frame_dimension['Dimension'].apply(clean_name)
)

df_narrative_frame_dimension['Column name dimension active'] = (
    "dimension_active_" +
    df_narrative_frame_dimension['Dimension'].apply(clean_name)
)

df_narrative_frame_dimension["Frequency passive"] = (
    df_narrative_frame_dimension["Column name dimension passive"]
    .apply(lambda col: df_segments_analysis[col].sum(skipna=True))
)

def sum_passive_given_frame(row):
    dim_col   = row["Column name dimension passive"]
    frame_col = row["Column name narrative frame passive"]

    return df_segments_analysis.loc[
        df_segments_analysis[frame_col] == 1,
        dim_col
    ].sum(skipna=True)

df_narrative_frame_dimension["Frequency passive extended"] = (
    df_narrative_frame_dimension
        .apply(sum_passive_given_frame, axis=1)
)

df_narrative_frame_dimension["Frequency active"] = (
    df_narrative_frame_dimension["Column name dimension active"]
    .apply(lambda col: df_segments_analysis[col].sum(skipna=True))
)

def sum_active_given_frame(row):
    dim_col   = row["Column name dimension active"]
    frame_col = row["Column name narrative frame active"]

    return df_segments_analysis.loc[
        df_segments_analysis[frame_col] == 1,
        dim_col
    ].sum(skipna=True)

df_narrative_frame_dimension["Frequency active extended"] = (
    df_narrative_frame_dimension
        .apply(sum_active_given_frame, axis=1)
)

df_narrative_frame_dimension["Percentage passive total"] = (
    df_narrative_frame_dimension["Column name dimension passive"]
    .apply(lambda col: df_segments_analysis[col].sum(skipna=True))
    / df_segments_analysis['victim_presence'].isin([1, 3]).sum()
    * 100
).round(2)

df_narrative_frame_dimension["Percentage active total"] = (
    df_narrative_frame_dimension["Column name dimension active"]
    .apply(lambda col: df_segments_analysis[col].sum(skipna=True))
    / df_segments_analysis['victim_presence'].isin([2, 3]).sum()
    * 100
).round(2)

def pct_passive_per_frame_clear(row):
    dim_col   = row["Column name dimension passive"]
    frame_col = row["Column name narrative frame passive"]

    # numerator: count of dimension
    num = df_segments_analysis.loc[
        df_segments_analysis[frame_col] == 1,
        dim_col
    ].sum(skipna=True)

    # denominator: records where this narrative frame is present
    den = (df_segments_analysis[frame_col] ==1).sum()

    if den == 0:
        return 0.0  # or np.nan if you prefer

    return num / den * 100

df_narrative_frame_dimension["Percentage passive per narrative frame clear"] = (
    df_narrative_frame_dimension
        .apply(pct_passive_per_frame_clear, axis=1)
        .round(2)
)

def pct_active_per_frame_clear(row):
    dim_col   = row["Column name dimension active"]
    frame_col = row["Column name narrative frame active"]

    # numerator: count of dimension
    num = df_segments_analysis.loc[
        df_segments_analysis[frame_col] == 1,
        dim_col
    ].sum(skipna=True)

    # denominator: records where this narrative frame is present
    den = (df_segments_analysis[frame_col] ==1).sum()

    if den == 0:
        return 0.0  # or np.nan if you prefer

    return num / den * 100

df_narrative_frame_dimension["Percentage active per narrative frame clear"] = (
    df_narrative_frame_dimension
        .apply(pct_active_per_frame_clear, axis=1)
        .round(2)
)

def pct_passive_per_frame_indirect(row):
    dim_col   = row["Column name dimension passive"]
    frame_col = row["Column name narrative frame passive"]

    # numerator: count of dimension
    num = df_segments_analysis.loc[
        df_segments_analysis[frame_col] == 0.5,
        dim_col
    ].sum(skipna=True)

    # denominator: records where this narrative frame is present
    den = (df_segments_analysis[frame_col] ==0.5).sum()

    if den == 0:
        return 0.0  # or np.nan if you prefer

    return num / den * 100

df_narrative_frame_dimension["Percentage passive per narrative frame indirect"] = (
    df_narrative_frame_dimension
        .apply(pct_passive_per_frame_indirect, axis=1)
        .round(2)
)

def pct_active_per_frame_indirect(row):
    dim_col   = row["Column name dimension active"]
    frame_col = row["Column name narrative frame active"]

    # numerator: count of dimension
    num = df_segments_analysis.loc[
        df_segments_analysis[frame_col] == 0.5,
        dim_col
    ].sum(skipna=True)

    # denominator: records where this narrative frame is present
    den = (df_segments_analysis[frame_col] ==0.5).sum()

    if den == 0:
        return 0.0  # or np.nan if you prefer

    return num / den * 100

df_narrative_frame_dimension["Percentage active per narrative frame indirect"] = (
    df_narrative_frame_dimension
        .apply(pct_active_per_frame_indirect, axis=1)
        .round(2)
)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.ticker import PercentFormatter

# Define plot function

def plot_victim_counts_percentage(data, title, filename, order):
    counts = data['victim_presence'].value_counts(dropna=False)
    total = counts.sum()

    # convert to percentages
    percentages = counts / total * 100
    ordered_percentages = [percentages.get(k, 0) for k in order]

    labels = []
    for k in order:
        if k == 0 or k == 0.0:
            labels.append("0 (no mention)")
        elif k == 1 or k == 1.0:
            labels.append("1 (spoken about)")
        elif k == 2 or k == 2.0:
            labels.append("2 (speaking)")
        elif k == 3 or k == 3.0:
            labels.append("3 (speaking and spoken about)")
        else:
            labels.append(str(k))

    plt.figure()
    plt.bar(range(len(ordered_percentages)), ordered_percentages)
    plt.xticks(range(len(ordered_percentages)), labels, rotation=0)

    plt.ylabel("Percentage (%)")
    plt.gca().yaxis.set_major_formatter(PercentFormatter())

    plt.title(title)
    plt.tight_layout()
    plt.savefig(filename, dpi=150, bbox_inches="tight")
    plt.show()


def plot_histograms(data, frame_cols,suffix, limit=6):
    """One histogram per frame column; bins aligned to 0, 0.5, 1."""
    for c in frame_cols[:limit]:
        plt.figure()
        bins = [0.25, 0.75, 1.25]
        plt.hist(data[c].dropna(), bins=bins, rwidth=0.9)
        plt.xticks([0.5, 1])
        plt.xlabel("Frame intensity")
        plt.ylabel("Count")
        plt.title(f"Distribution of {c}{suffix}")
        plt.tight_layout()
        plt.savefig(f"hist_{c}{suffix}.png", dpi=150, bbox_inches="tight")
        plt.show()

# Visualizations

plot_victim_counts_percentage(
    df_segments_analysis,
    title="Victim presence — Percentage of segments",
    filename="victim_presence_counts.png",
    order=[1, 2, 3],
)


In [ ]:
frame_cols_active = ['narrative_frame_citizenship_active','narrative_frame_suffering_active','narrative_frame_agency_active','narrative_frame_reconciliation_active','narrative_frame_victimblaming_active','narrative_frame_angerhatred_active']
frame_cols_passive = ['narrative_frame_citizenship_passive','narrative_frame_suffering_passive','narrative_frame_agency_passive','narrative_frame_reconciliation_passive','narrative_frame_victimblaming_passive','narrative_frame_angerhatred_passive']

df_segments_analysis_clear_narrative_frames = df_segments_analysis[df_segments_analysis['victim_presence'].isin([1, 2, 3])].replace(0.5, 0)
df_segments_analysis_reasonable_narrative_frames = df_segments_analysis[df_segments_analysis['victim_presence'].isin([1, 2, 3])].replace(1, 0)
df_segments_analysis_reasonable_narrative_frames = df_segments_analysis_reasonable_narrative_frames.replace(0.5, 1)

plot_labels_passive = {
    "narrative_frame_suffering_passive": "Suffering",
    "narrative_frame_citizenship_passive": "Citizenship",
    "narrative_frame_agency_passive": "Agency",
    "narrative_frame_reconciliation_passive": "Reconciliation",
    "narrative_frame_victimblaming_passive": "Victim blaming",
    "narrative_frame_angerhatred_passive": "Emotion"
}

plot_labels_active = {
    "narrative_frame_suffering_active": "Suffering",
    "narrative_frame_citizenship_active": "Citizenship",
    "narrative_frame_agency_active": "Agency",
    "narrative_frame_reconciliation_active": "Reconciliation",
    "narrative_frame_victimblaming_active": "Victim blaming",
    "narrative_frame_angerhatred_active": "Emotion"
}

def plot_frame_means(data, frame_cols, title, filename, label_map=None):
    df_num = data.loc[:, frame_cols].copy()

    means = df_num.mean(skipna=True).mul(100).sort_values(ascending=True)

    plt.figure(figsize=(8, 5))
    bars = plt.barh(range(len(means)), means.values)

    if label_map is not None:
        yt_labels = [label_map.get(col, col) for col in means.index]
    else:
        yt_labels = means.index

    plt.yticks(range(len(means)), yt_labels)
    plt.xlabel("Frame presence (%)")
    plt.xlim(0, 100)
    plt.title(title)

    # Place label just to the right of each bar
    for rect, value in zip(bars, means.values):
        width = rect.get_width()
        plt.text(
            width + 0.8,          # small offset from bar end
            rect.get_y() + rect.get_height() / 2,
            f"{value:.1f}%",
            va="center",
            ha="left",
            fontsize=9
        )

    plt.tight_layout()
    plt.savefig(filename, dpi=150, bbox_inches="tight")
    plt.show()
    
plot_frame_means(
    df_segments_analysis_clear_narrative_frames[df_segments_analysis_clear_narrative_frames['victim_presence'].isin([1,3])],
    frame_cols_passive,
    title="Percentage of clear presence of narrative frame across segments where victims are spoken about",
    filename="narrative_frames_clear_passive.png",
    label_map=plot_labels_passive
    )

plot_frame_means(
    df_segments_analysis_clear_narrative_frames[df_segments_analysis_clear_narrative_frames['victim_presence'].isin([2,3])],
    frame_cols_active,
    title="Percentage of clear presence of narrative frame across segments where victims are speaking",
    filename="narrative_frames_clear_active.png",
    label_map=plot_labels_active
    )

plot_frame_means(
    df_segments_analysis_reasonable_narrative_frames[df_segments_analysis_reasonable_narrative_frames['victim_presence'].isin([1,3])],
    frame_cols_passive,
    title="Percentage of an indirect presence of narrative frame across segments where victims are spoken about",
    filename="narrative_frames_indirect_passive.png",
    label_map=plot_labels_passive
    )

plot_frame_means(
    df_segments_analysis_reasonable_narrative_frames[df_segments_analysis_reasonable_narrative_frames['victim_presence'].isin([2,3])],
    frame_cols_active,
    title="Percentage of an indirect presence of narrative frame across segments where victims are speaking",
    filename="narrative_frames_indirect_active.png",
    label_map=plot_labels_active
    )

In [ ]:
frame_cols_all = ['narrative_frame_citizenship','narrative_frame_suffering','narrative_frame_agency','narrative_frame_reconciliation','narrative_frame_victimblaming','narrative_frame_angerhatred']
df_segments_analysis[frame_cols_all]=df_segments_analysis[frame_cols_all].replace(0.5, 0)
plot_labels = {
    "narrative_frame_suffering": "Suffering",
    "narrative_frame_citizenship": "Citizenship",
    "narrative_frame_agency": "Agency",
    "narrative_frame_reconciliation": "Reconciliation",
    "narrative_frame_victimblaming": "Victim blaming",
    "narrative_frame_angerhatred": "Stigmatized emotion"
    }

def plot_heatmap_by_victim(data, frame_cols, title, filename, order, label_map=None):
    grouped = data.groupby('victim_presence')[frame_cols].mean().reindex(order)
    plt.figure()
    plt.imshow(grouped.values, aspect="auto")

    xlabels = [plot_labels.get(col, col) for col in frame_cols]
    plt.xticks(
        ticks=np.arange(len(frame_cols)),
        labels=xlabels,
        rotation=45,
        ha="right"
    )
    # Build ylabels from order
    ylabels = []
    for k in order:
        if k == 0 or k == 0.0:
            ylabels.append("0 (no mention)")
        elif k == 1 or k == 1.0:
            ylabels.append("1 (spoken about)")
        elif k == 2 or k == 2.0:
            ylabels.append("2 (speaking)")
        elif k == 3 or k == 3.0:
            ylabels.append("3 (speaking and spoken about)")
        else:
            ylabels.append(str(k))
    plt.yticks(ticks=np.arange(len(order)), labels=ylabels)
    plt.colorbar(label="Frame presence (%)")
    plt.title(title)
    plt.tight_layout()
    plt.savefig( filename, dpi=150, bbox_inches="tight")
    plt.show()

plot_heatmap_by_victim(
    df_segments_analysis,
    frame_cols_all,
    title="Narrative frames by victim presence (victims only)",
    filename="frames_by_victim_presence_heatmap_filtered.png",
    order=[1.0, 2.0,3.0],
    label_map = plot_labels
    )

print(df_segments_analysis.groupby('victim_presence')[frame_cols].mean())

In [24]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

# Columns to use (adjust names if needed)
frame_cols = [
    "narrative_frame_suffering",
    "narrative_frame_citizenship",
    "narrative_frame_agency",
    "narrative_frame_reconciliation",
    "narrative_frame_victimblaming",
    "narrative_frame_angerhatred",
]

plot_labels = {
    "narrative_frame_suffering": "Suffering",
    "narrative_frame_citizenship": "Citizenship",
    "narrative_frame_agency": "Agency",
    "narrative_frame_reconciliation": "Reconciliation",
    "narrative_frame_victimblaming": "Victim blaming",
    "narrative_frame_angerhatred": "Emotion"
    }

xlabels = [plot_labels.get(col, col) for col in frame_cols]

# Keep only those columns and turn "1" into 1, everything else into 0
# (assumes df_segments_analysis already has 0, 0.5, 1 as floats)
presence_matrix = (df_segments_analysis[frame_cols] == 1).astype(int).values

# Make a white vs colored colormap
cmap = ListedColormap(["white", "#AF2543"])

plt.figure(figsize=(8, 10))
plt.imshow(
    presence_matrix,
    aspect="auto",
    cmap=cmap,
    interpolation="nearest",
    origin="upper",  # first row at the top
)
plt.xticks(
    ticks=np.arange(len(frame_cols)),
    labels=xlabels,
    rotation=45,
    ha="right"
)

plt.yticks([])  # hide segment tick labels (too many); remove this line if you want them

plt.xlabel("Narrative frame")
plt.ylabel("Segments in document (top → bottom)")
plt.title("Locations of narrative frames (value = 1) across truth commission document")

plt.tight_layout()
plt.show()

# Comparison between CEH, REHMI and Actoras (TEQLEA)

In [2]:
df_segments_analysis_CEH = pd.read_parquet("df_segments_analysis_CEH.parquet")
df_segments_analysis_REMHI = pd.read_parquet("df_segments_analysis_REHMI.parquet")
df_segments_analysis_TEQLEA = pd.read_parquet("df_segments_analysis_TEQLEA.parquet")

In [25]:
import matplotlib.font_manager as fm

sorted({fm.FontProperties(fname=f).get_name() 
        for f in fm.findSystemFonts()
        if "Avenir" in f})

import matplotlib.pyplot as plt

plt.rcParams["font.family"] = ["Avenir", "sans-serif"]
plt.rcParams["font.size"] = 11

In [ ]:
frame_cols_active = ['narrative_frame_citizenship_active','narrative_frame_suffering_active','narrative_frame_agency_active','narrative_frame_reconciliation_active','narrative_frame_angerhatred_active']
frame_cols_passive = ['narrative_frame_citizenship_passive','narrative_frame_suffering_passive','narrative_frame_agency_passive','narrative_frame_reconciliation_passive','narrative_frame_angerhatred_passive']
cols_to_replace = frame_cols_active + frame_cols_passive

df_segments_analysis_clear_narrative_frames_CEH = (
    df_segments_analysis_CEH
    .loc[df_segments_analysis_CEH["victim_presence"].isin([1, 2, 3])]
    .copy()
)
df_segments_analysis_clear_narrative_frames_CEH[cols_to_replace] = (
    df_segments_analysis_clear_narrative_frames_CEH[cols_to_replace]
    .replace(0.5, 0)
)
df_segments_analysis_indirect_narrative_frames_CEH  = (
    df_segments_analysis_CEH
    .loc[df_segments_analysis_CEH["victim_presence"].isin([1, 2, 3])]
    .copy()
)
df_segments_analysis_indirect_narrative_frames_CEH[cols_to_replace] = (
    df_segments_analysis_indirect_narrative_frames_CEH[cols_to_replace]
    .replace(1, 0)
    .replace(0.5, 1)
)

df_segments_analysis_clear_narrative_frames_REMHI = (
    df_segments_analysis_REMHI
    .loc[df_segments_analysis_REMHI["victim_presence"].isin([1, 2, 3])]
    .copy()
)
df_segments_analysis_clear_narrative_frames_REMHI[cols_to_replace] = (
    df_segments_analysis_clear_narrative_frames_REMHI[cols_to_replace]
    .replace(0.5, 0)
)
df_segments_analysis_indirect_narrative_frames_REMHI  = (
    df_segments_analysis_REMHI
    .loc[df_segments_analysis_REMHI["victim_presence"].isin([1, 2, 3])]
    .copy()
)
df_segments_analysis_indirect_narrative_frames_REMHI[cols_to_replace] = (
    df_segments_analysis_indirect_narrative_frames_REMHI[cols_to_replace]
    .replace(1, 0)
    .replace(0.5, 1)
)

df_segments_analysis_clear_narrative_frames_TEQLEA = (
    df_segments_analysis_TEQLEA
    .loc[df_segments_analysis_TEQLEA["victim_presence"].isin([1, 2, 3])]
    .copy()
)
df_segments_analysis_clear_narrative_frames_TEQLEA[cols_to_replace] = (
    df_segments_analysis_clear_narrative_frames_TEQLEA[cols_to_replace]
    .replace(0.5, 0)
)
df_segments_analysis_indirect_narrative_frames_TEQLEA  = (
    df_segments_analysis_TEQLEA
    .loc[df_segments_analysis_TEQLEA["victim_presence"].isin([1, 2, 3])]
    .copy()

df_segments_analysis_indirect_narrative_frames_TEQLEA[cols_to_replace] = (
    df_segments_analysis_indirect_narrative_frames_TEQLEA[cols_to_replace]
    .replace(1, 0)
    .replace(0.5, 1)
)

plot_labels_passive = {
    "narrative_frame_suffering_passive": "Suffering",
    "narrative_frame_citizenship_passive": "Citizenship",
    "narrative_frame_agency_passive": "Agency",
    "narrative_frame_reconciliation_passive": "Reconciliation",
    "narrative_frame_victimblaming_passive": "Victim blaming",
    "narrative_frame_angerhatred_passive": "Stigmatized emotion"
}

plot_labels_passive = {
    "narrative_frame_suffering_active": "Suffering",
    "narrative_frame_citizenship_active": "Citizenship",
    "narrative_frame_agency_active": "Agency",
    "narrative_frame_reconciliation_active": "Reconciliation",
    "narrative_frame_victimblaming_active": "Victim blaming",
    "narrative_frame_angerhatred_active": "Stigmatized emotion"
}


### Compare victim-protagonist representation

In [45]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

def plot_victim_counts_percentage_compare(
    data_ceh,
    data_remhi,
    data_teqlea,   
    title,
    filename,
    order
):
    # --- CEH percentages ---
    counts_ceh = data_ceh["victim_presence"].value_counts(dropna=False)
    total_ceh = counts_ceh.sum()
    perc_ceh = counts_ceh / total_ceh * 100

    # --- REMHI percentages ---
    counts_remhi = data_remhi["victim_presence"].value_counts(dropna=False)
    total_remhi = counts_remhi.sum()
    perc_remhi = counts_remhi / total_remhi * 100

    # --- TEQLEA percentages ---  
    counts_teqlea = data_teqlea["victim_presence"].value_counts(dropna=False)
    total_teqlea = counts_teqlea.sum()
    perc_teqlea = counts_teqlea / total_teqlea * 100

    # order all according to `order`
    ordered_ceh = [perc_ceh.get(k, 0) for k in order]
    ordered_rehmi = [perc_remhi.get(k, 0) for k in order]
    ordered_teqlea = [perc_teqlea.get(k, 0) for k in order] 

    # x labels
    labels = []
    for k in order:
        if k == 0 or k == 0.0:
            labels.append("No mention")
        elif k == 1 or k == 1.0:
            labels.append("Spoken about")
        elif k == 2 or k == 2.0:
            labels.append("Speaking")
        elif k == 3 or k == 3.0:
            labels.append("Speaking and spoken about")
        else:
            labels.append(str(k))

    x = np.arange(len(order))
    width = 0.25  

    plt.figure()

    # CEH bars
    bars_ceh = plt.bar(
        x - width,
        ordered_ceh,
        width=width,
        color="#20426B",
        label="CEH"
    )

    # REMHI bars
    bars_remhi = plt.bar(
        x,
        ordered_rehmi,
        width=width,
        color="#AF2543",
        label="REMHI"
    )

    # TEQLEA bars  
    bars_teqlea = plt.bar(
        x + width,
        ordered_teqlea,
        width=width,
        color="#2A7F62",   
        label="Actoras"
    )

    plt.xticks(x, labels, rotation=0)
    plt.ylabel("Percentage (%)")
    plt.gca().yaxis.set_major_formatter(PercentFormatter())
    plt.title(title)
    plt.legend()

    for bars in [bars_ceh, bars_remhi, bars_teqlea]:  
        for bar in bars:
            height = bar.get_height()
            plt.text(
                bar.get_x() + bar.get_width() / 2,
                height + 0.5,
                f"{height:.1f}%",
                ha="center",
                va="bottom",
                fontsize=9
            )

    plt.tight_layout()
    plt.savefig(filename, dpi=150, bbox_inches="tight")
    plt.show()

In [ ]:
plot_victim_counts_percentage_compare(
    df_segments_analysis_CEH,
    df_segments_analysis_REMHI,
    df_segments_analysis_TEQLEA,
    title="Victim representation (CEH vs REMHI vs Actoras)",
    filename="victim_presence_CEH_REMHI_Actoras.png",
    order=[1, 2, 3]
)

### Compare narrative frame presence

In [51]:
import numpy as np
import matplotlib.pyplot as plt

def plot_frame_means_comparison(
    data_ceh,
    data_remhi,
    data_teqlea,      
    frame_cols,
    title,
    filename,
    label_map=None,
    color_ceh="#20426B",     # dark blue
    color_remhi="#AF2543",   # dark red
    color_teqlea="#2A7F62"   #  green
):
    # --- compute means (percentages) ---
    means_ceh = (
        data_ceh.loc[:, frame_cols]
        .mean(skipna=True)
        .mul(100)
    )
    means_remhi = (
        data_remhi.loc[:, frame_cols]
        .mean(skipna=True)
        .mul(100)
    )
    means_teqlea = (                                 
        data_teqlea.loc[:, frame_cols]
        .mean(skipna=True)
        .mul(100)
    )

    # sort by CEH (or choose REHMI if you prefer)
    order = means_remhi.sort_values(ascending=True).index
    means_ceh = means_ceh.loc[order]
    means_remhi = means_remhi.loc[order]
    means_teqlea = means_teqlea.loc[order]           

    # labels
    if label_map is not None:
        yt_labels = [label_map.get(col, col) for col in order]
    else:
        yt_labels = order

    # --- plotting ---
    y = np.arange(len(order))
    bar_height = 0.25   # CHANGED: thinner bars for 3 datasets

    plt.figure(figsize=(8, 5))

    bars_ceh = plt.barh(
        y + bar_height,
        means_ceh.values,
        height=bar_height,
        color=color_ceh,
        label="CEH"
    )

    bars_rehmi = plt.barh(
        y,
        means_remhi.values,
        height=bar_height,
        color=color_remhi,
        label="REMHI"
    )

    bars_teqlea = plt.barh(                            
        y - bar_height,
        means_teqlea.values,
        height=bar_height,
        color=color_teqlea,
        label="Actoras"
    )

    plt.yticks(y, yt_labels)
    plt.xlabel("Frame presence (%)")
    plt.xlim(0, 100)
    plt.title(title)
    plt.legend(handles=[bars_ceh, bars_rehmi, bars_teqlea],   
               labels=["CEH", "REMHI", "Actoras"],
               loc="lower right")

    # --- annotate values ---
    for bars, values in [
        (bars_ceh, means_ceh.values),
        (bars_rehmi, means_remhi.values),
        (bars_teqlea, means_teqlea.values)            
    ]:
        for rect, value in zip(bars, values):
            plt.text(
                rect.get_width() + 0.8,
                rect.get_y() + rect.get_height() / 2,
                f"{value:.1f}%",
                va="center",
                ha="left",
                fontsize=9
            )

    plt.tight_layout()
    plt.savefig(filename, dpi=150, bbox_inches="tight")
    plt.show()

In [ ]:
plot_frame_means_comparison(
    df_segments_analysis_clear_narrative_frames_CEH[
        df_segments_analysis_clear_narrative_frames_CEH['victim_presence'].isin([1, 3])
    ],
    df_segments_analysis_clear_narrative_frames_REMHI[
        df_segments_analysis_clear_narrative_frames_REMHI['victim_presence'].isin([1, 3])
    ],
    df_segments_analysis_clear_narrative_frames_TEQLEA[
        df_segments_analysis_clear_narrative_frames_TEQLEA['victim_presence'].isin([1, 3])
    ],
    frame_cols_passive,
    title="Direct narrative frame prevalence where victims are spoken about",
    filename="narrative_frames_passive_CEH_REHMI_Actoras.png",
    label_map=plot_labels_passive
)

plot_frame_means_comparison(
    df_segments_analysis_clear_narrative_frames_CEH[
        df_segments_analysis_clear_narrative_frames_CEH['victim_presence'].isin([2, 3])
    ],
    df_segments_analysis_clear_narrative_frames_REMHI[
        df_segments_analysis_clear_narrative_frames_REMHI['victim_presence'].isin([2, 3])
    ],
    df_segments_analysis_clear_narrative_frames_TEQLEA[
        df_segments_analysis_clear_narrative_frames_TEQLEA['victim_presence'].isin([2, 3])
    ],
    frame_cols_active,
    title="Direct narrative frame prevalence where victims are speaking",
    filename="narrative_frames_active_CEH_REMHI_Actoras.png",
    label_map=plot_labels_active
)

plot_frame_means_comparison(
    df_segments_analysis_indirect_narrative_frames_CEH[
        df_segments_analysis_indirect_narrative_frames_CEH['victim_presence'].isin([1, 3])
    ],
    df_segments_analysis_indirect_narrative_frames_REMHI[
        df_segments_analysis_indirect_narrative_frames_REMHI['victim_presence'].isin([1, 3])
    ],
    df_segments_analysis_indirect_narrative_frames_TEQLEA[
        df_segments_analysis_indirect_narrative_frames_TEQLEA['victim_presence'].isin([1, 3])
    ],
    frame_cols_passive,
    title="Indirect narrative frame prevalence where victims are spoken about",
    filename="narrative_frames_passive_CEH_REMHI_Actoras.png",
    label_map=plot_labels_passive
)

plot_frame_means_comparison(
    df_segments_analysis_indirect_narrative_frames_CEH[
        df_segments_analysis_indirect_narrative_frames_CEH['victim_presence'].isin([2, 3])
    ],
    df_segments_analysis_indirect_narrative_frames_REMHI[
        df_segments_analysis_indirect_narrative_frames_REMHI['victim_presence'].isin([2, 3])
    ],
    df_segments_analysis_indirect_narrative_frames_TEQLEA[
        df_segments_analysis_indirect_narrative_frames_TEQLEA['victim_presence'].isin([2, 3])
    ],
    frame_cols_active,
    title="Indirect narrative frame prevalence where victims are speaking",
    filename="narrative_frames_active_CEH_REMHI_Actoras.png",
    label_map=plot_labels_active
)